# "SQL"
> "Example database to learn and review SQL"

- author: Christopher Thiemann
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [SQL, ]
- hide: false
- search_exclude: true

In [1]:
#hide
import warnings


import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols


import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

# load specific python package
import sqlite3

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!curl https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -o database.zip
!unzip database.zip
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0   972k      0 --:--:-- --:--:-- --:--:--  972k
Archive:  database.zip
  inflating: chinook.db              
chinook.db  database.zip  sample_data


In [4]:
con = sqlite3.connect("chinook.db")
cursor = con.cursor()
with con:
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table_name in cursor.fetchall():
        
        print(table_name[0])
        print(pd.read_sql_query(f"select * from {table_name[0]}", con).head(), "\n\n\n\n\n\n")

albums
   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3 






sqlite_sequence
          name   seq
0       genres    25
1  media_types     5
2      artists   275
3       albums   347
4       tracks  3503 






artists
   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains 






customers
   CustomerId  FirstName     LastName  \
0           1       Luís    Gonçalves   
1           2     Leonie       Köhler   
2           3   François     Tremblay   
3           4      Bjørn       Hansen   
4           5  František  Wichterlová   

                   

## Basic Queries

In [37]:
pd.read_sql_query("select * from employees limit 10", con)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [45]:
query = """
select *, CASE
		WHEN EmployeeId > 1 THEN
			'short'
		WHEN EmployeeId > 5 THEN 'medium'
		ELSE
			'long'
		END category
         from employees limit 10
"""

pd.read_sql_query(query, con)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email,category
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com,long
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com,short
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com,short
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com,short
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,short
5,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com,short
6,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com,short
7,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com,short


In [39]:
pd.read_sql_query("select * from employees limit 10 offset 3", con)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
1,5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
2,6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
3,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
4,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [ ]:
pd.read_sql_query("select * from artists", con, index_col='ArtistId')

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains
...,...
271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
272,Emerson String Quartet
273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."


In [51]:
pd.read_sql_query("select * from artists inner join albums on artists.ArtistId = albums.ArtistId", con)

,ArtistId,Name,AlbumId,Title,ArtistId
0,1,AC/DC,1,For Those About To Rock We Salute You,1
1,2,Accept,2,Balls to the Wall,2
2,2,Accept,3,Restless and Wild,2
3,1,AC/DC,4,Let There Be Rock,1
4,3,Aerosmith,5,Big Ones,3
...,...,...,...,...,...
342,226,Eugene Ormandy,343,Respighi:Pines of Rome,226
343,272,Emerson String Quartet,344,Schubert: The Late String Quartets & String Qu...,272
344,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",345,Monteverdi: L'Orfeo,273
345,274,Nash Ensemble,346,Mozart: Chamber Music,274


In [17]:
pd.read_sql_query("select * from media_types where Name like '5AAC%'", con)

,MediaTypeId,Name


In [18]:
pd.read_sql_query("select * from playlists where name like '_usic'", con)

,PlaylistId,Name
0,1,Music
1,8,Music


## Aggregate Functions
COUNT() SUM() MAX() MIN() AVG()


![](https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/sqlite-sample-database-diagram-color.pdf)

## Helper Functions

## Plot for the Blog Post

## Sources

- Hello This is a markdown page {% cite signaltrain %}
- http://openbook.rheinwerk-verlag.de/python/33_003.html#u33.3.1
- https://www.sqlitetutorial.net/sqlite-left-join/
- https://realpython.com/python-sqlite-sqlalchemy/

- https://docs.snowflake.net/manuals/sql-reference-commands.html

- https://www.geeksforgeeks.org/sql-tutorial/

- https://github.com/harelba/q

- https://github.com/kennethreitz-archive/records

- https://github.com/simonw/csvs-to-sqlite

- https://github.com/prestodb/presto

- https://sqlzoo.net/

- https://www.sqlstyle.guide/

- https://twitter.com/CMastication/status/1289919608309329921

- https://blazingsql.com/

- https://duckdb.org/

- https://github.com/nackjicholson/aiosql

- https://github.com/sql-machine-learning/sqlflow

- https://github.com/ibis-project/ibis


## References

{% bibliography --cited %}